In [1]:
from google.colab import files

# Upload Resume.csv from your computer
uploaded = files.upload()


Saving Resume.csv to Resume.csv


In [2]:
!pip install transformers


In [3]:
import pandas as pd
import re
from transformers import pipeline
from tqdm import tqdm


In [4]:
df = pd.read_csv("Resume.csv")
df.columns = df.columns.str.strip()  # Clean column names


In [5]:
def clean_text(text):
    if pd.isnull(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'\s+', ' ', text)           # remove extra spaces and newlines
    text = re.sub(r'[^\w\s]', '', text)        # remove punctuation
    return text.strip()

df['CleanResume'] = df['Resume_str'].apply(clean_text)


In [6]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [7]:
candidate_labels = ["Data Science", "HR", "DevOps", "Testing",
                    "Design", "Software Engineer", "Business Analyst"]


In [8]:
df = df.head(100)

predicted_labels = []

for resume in tqdm(df['CleanResume']):
    if not resume.strip():
        predicted_labels.append("Unknown")
        continue
    result = classifier(resume, candidate_labels)
    predicted_labels.append(result['labels'][0])  # top predicted label


100%|██████████| 100/100 [2:23:32<00:00, 86.13s/it]


In [9]:
df['PredictedCategory_GenAI'] = predicted_labels

df.to_csv("GenAI_Classified_Resumes.csv", index=False)

from google.colab import files
files.download("GenAI_Classified_Resumes.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>